In [11]:
import jams
import glob

import instrument_recognition as ir

# load data path
dataset = 'medleydb'
partition = 'train'
soundscape_name = 'mdb-synthetic-mono-OVERLAP'
data_path = ir.DATA_DIR / dataset / partition / soundscape_name
all_jams_files = glob.glob(str(data_path / '*.jams'), recursive=True)


In [12]:
records = [jams.load(p) for p in all_jams_files]

In [70]:
classlist = os.listdir(data_path.parent / 'foreground' )
classlist

['auxiliary percussion',
 'cello',
 'vocalists',
 'double bass',
 'acoustic guitar',
 'viola section',
 'female singer',
 'tack piano',
 'violin',
 'distorted electric guitar',
 'brass section',
 'electric bass',
 'vibraphone',
 'male singer',
 'oboe',
 'viola',
 'violin section',
 'clean electric guitar',
 'trumpet',
 'drum set',
 'synthesizer',
 'piano']

In [66]:
example_jam = records[0]
get_scaper_observations(example_jam)[0].value

{'label': 'male singer',
 'source_file': '/home/hugo/lab/instrument-recognition/data/medleydb/train/foreground/male singer/Meaxic_YouListen-1-male singer.wav',
 'source_time': 26.224890859771936,
 'event_time': 0.0,
 'event_duration': 3.2899817759553738,
 'snr': 13.918368980280475,
 'role': 'foreground',
 'pitch_shift': 2.2979419414087676,
 'time_stretch': 1.0181099059809644}

In [91]:
import numpy as np 
def get_scaper_observations(jam):
    return jam.search(namespace='scaper')[0]['data']

def get_path_to_audio(jam):
    obvs = get_scaper_observations(jam)
    return obvs[0].value['source_file']

def get_simple_annotation(obv):
    """ for a jams Observation, returns a dict
    with label, start_time and end_time
    """
    return dict(label=obv.value['label'],
        start_time=obv.value['event_time'],
        end_time=start_time+obv.value['event_duration']) 

def quantize_ceil(value, numerator, denominator, num_decimals=4):
    ratio = (numerator/denominator)
    return round((value // ratio + 1) * ratio, num_decimals)

def quantize_floor(value, numerator, denominator, num_decimals=4):
    ratio = (numerator/denominator)
    return round(value // ratio * ratio, num_decimals)

def get_one_hot_matrix(jam, classlist: list, resolution: float = 1.0):
    # get duration from file metadata
    duration = jam.file_metadata.duration
    obvs = get_scaper_observations(jam)

    # determine the number of bins in the time axis
    assert duration % resolution == 0, \
        f'resolution {resolution} is not divisible by audio duration: {duration}'
    num_time_bins = int(duration // resolution)

    # make an empty matrix shape (time, classes)
    one_hot = np.zeros((num_time_bins, len(classlist)))
    time_axis = list(np.arange(0.0, duration, resolution))

    # get the indices for each label
    for obv in obvs:
        ann = obv.value

        start_time = ann['event_time']
        end_time = start_time + ann['event_duration']

        start_idx = time_axis.index(quantize_floor(start_time, resolution, duration))
        end_idx = time_axis.index(quantize_ceil(end_time, resolution, duration))

        label_idx = classlist.index(ann['label'])

        # now, index
        one_hot[start_idx:end_idx, label_idx] = 1
    



In [92]:
get_path_to_audio(example_jam)
print(get_one_hot_matrix(example_jam, classlist))

ValueError: 3.3 is not in list